In [1]:
import sys
import os
import subprocess
import numpy as np
from datetime import datetime, timedelta

sys.path.append('../sdpm_py_util')
import init_funs_forecast as initfuns
sys.path.append('/home/mspydell/models/PFM_root/PFM/driver')
#from driver_run_forecast_LV1234 import driver_run_fore_LV1234
from driver_run_pfm_phm import driver_run_pfm_phm


/home/mspydell/models/PFM_root/PFM/sdpm_py_util/ocn_funs_forecast.py:32: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater


In [ ]:
infile = '/home/mspydell/models/PFM_root/PFM/sdpm_py_util/pfm_input_file.py'
pkl_fnm = '/scratch/PFM_Simulations/forecast_pfile.pkl'


t00 = datetime.now()
# upon initialization, make the model_info.pkl file
initfuns.initialize_model( infile , pkl_fnm )

# print info from pickle file
initfuns.print_initial_model_info( pkl_fnm )


In [ ]:

driver_run_fore_LV1234( infile, pkl_fnm )

# to run from the command line do:
# python driver_run_hind_LV123.py '/home/mspydell/models/PFM_root/PFM/sdpm_py_util/phm_model_info_devel.py' '/scratch/PHM_Simulations/hindcast_pfile.pkl' > hindcast.out


In [2]:
#run = 'hind'
run = 'fore'
if run == 'hind': # hindcasting works with this newer method (6/10/25). make sure you are in PHM-env
    infile = '/home/mspydell/models/PFM_root/PFM/sdpm_py_util/phm_model_info_devel.py'
    pkl_fnm = '/scratch/PHM_Simulations/hindcast_pfile.pkl'
else:
    infile = '/home/mspydell/models/PFM_root/PFM/sdpm_py_util/pfm_input_file.py'
    pkl_fnm = '/scratch/PFM_Simulations/forecast_pfile.pkl'

driver_run_pfm_phm( infile, pkl_fnm )


first, we need all the model information.
from /home/mspydell/models/PFM_root/PFM/sdpm_py_util/pfm_input_file.py
we create a dictionary of model info
the model_info dictionary is saved to /scratch/PFM_Simulations/forecast_pfile.pkl
this file dictates how the model is run, where files are saved, etc.
------------------------------------------------------------
                        Model Set Up                        
------------------------------------------------------------
forecast                                                    
from           : 2025-06-10 12:00                           
to             : 2025-06-15 12:00                           
in a           : 5.0 day chunk                              
atm model      : ecmwf                                      
ocean_model    : hycom_new                                  
running levels : LV1, LV2, LV3, LV4                         
starting LV1
done with LV1
LV fore ran correctly? 2 (0=yes)
this took 
starting LV2
done 

python: can't open file '/home/mspydell/models/PFM_root/PFM/sdpm_py_util/driver_functions.py': [Errno 2] No such file or directory
python: can't open file '/home/mspydell/models/PFM_root/PFM/sdpm_py_util/driver_functions.py': [Errno 2] No such file or directory
python: can't open file '/home/mspydell/models/PFM_root/PFM/sdpm_py_util/driver_functions.py': [Errno 2] No such file or directory
python: can't open file '/home/mspydell/models/PFM_root/PFM/sdpm_py_util/driver_functions.py': [Errno 2] No such file or directory
